In [1]:
//TODO: Make changes to all the tags with : and replace with |

package service.sec

import org.jsoup.nodes.Element
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import org.slf4j.LoggerFactory

class ElementParser {
    private val logger = LoggerFactory.getLogger(ElementParser::class.java)

    fun parseElement(soup: Element, element: Element, checkIsSubEntity: Boolean = true): Map<String, Any?> {
        val elementName = element.tagName().split("|").last().lowercase()
        try {
            if (!element.hasAttr("contextref")) {
                logger.warn("Element $elementName has no contextref")
                return emptyMap()
            }

            val contextElement = soup.getElementById(element.attr("contextref"))
            if (checkIsSubEntity && contextElement?.getElementsByTag("xbrli|segment")?.isNotEmpty() == true) {
                logger.info("Element $elementName has a sub entity")
                return emptyMap()
            }

            val elementDict = mutableMapOf<String, Any?>()
            elementDict["name"] = element.tagName().split("|").last().lowercase()
            elementDict["value"] = cleanValue(element.text())
            elementDict["unit"] = retrieveUnit(soup, element)
            elementDict["date"] = retrieveDate(soup, element)

            if (elementDict["value"] == "") {
                elementDict["value"] = retrieveFromContext(soup, element.attr("contextref"))
            }

            if (elementDict["unit"] != "NA" && elementDict["value"] is String) {
                elementDict["value"] = cleanValue(elementDict["value"] as String)
            }

            if (element.hasAttr("sign") && element.attr("sign") == "-") {
                elementDict["value"] = -(elementDict["value"] as Double)
            }

            return elementDict
        } catch (e: Exception) {
            logger.error("Exception parsing $elementName: ${e.message}")
            return emptyMap()
        }
    }

    fun cleanValue(string: String): Any {
        if (string.trim() == "-") {
            return 0.0
        }

        return try {
            string.trim().replace(",", "").replace(" ", "").toDouble()
        } catch (e: Exception) {
            string
        }
    }

    fun retrieveUnit(soup: Element, element: Element): String {
        return try {
            soup.getElementById(element.attr("unitref"))?.text()?.trim() ?: element.attr("unitref")
        } catch (e: Exception) {
            "NA"
        }
    }

    fun retrieveDate(soup: Element, element: Element): String {
        val dateTagList = listOf("xbrli|enddate", "xbrli|instant", "xbrli|period", "enddate", "instant")

        val contextElement = soup.getElementById(element.attr("contextref"))
        logger.info("Context element: $contextElement")

        if (contextElement != null) {
            val periodElement = contextElement?.selectFirst("xbrli|period")//.firstOrNull()
            logger.info("Period element: $periodElement")

            if (periodElement != null) {
                dateTagList.forEach { tag ->
                    val dateElement = periodElement.selectFirst(tag)//.firstOrNull()
                    val dateVal = dateElement?.text()?.trim()
                    logger.info("Found date value '$dateVal' for tag '$tag'")

                    if (!dateVal.isNullOrEmpty()) {
                        return try {
                            LocalDate.parse(dateVal, DateTimeFormatter.ISO_DATE).toString()
                        } catch (e: Exception) {
                            logger.error("Error parsing date value '$dateVal' for tag '$tag': ${e.message}")
                            "NA"
                        }
                    }
                }
            } else {
                // Check if it's an instant date
                val instantElement = contextElement.getElementsByTag("xbrli|instant").firstOrNull()
                val instantDateVal = instantElement?.text()?.trim()
                logger.info("Found instant date value '$instantDateVal'")

                if (!instantDateVal.isNullOrEmpty()) {
                    return try {
                        LocalDate.parse(instantDateVal, DateTimeFormatter.ISO_DATE).toString()
                    } catch (e: Exception) {
                        logger.error("Error parsing instant date value '$instantDateVal': ${e.message}")
                        "NA"
                    }
                }
            }
        }

        return try {
            LocalDate.parse(element.attr("contextref"), DateTimeFormatter.ISO_DATE).toString()
        } catch (e: Exception) {
            logger.error("Error parsing contextref as date: ${e.message}")
            "NA"
        }
    }

    fun retrieveFromContext(soup: Element, contextRef: String): String {
        return try {
            val context = soup.getElementsByTag("xbrli|context").first { it.id() == contextRef }
            context.getElementsByTag("xbrldi|explicitmember").text().split("|").last().trim()
        } catch (e: Exception) {
            logger.debug("Failed to retrieve from context: ${e.message}")
            ""
        }
    }
}


In [2]:
val eP = Line_1_jupyter.ElementParser()

In [3]:
// Fetch and parse the SEC data
val url = "https://www.sec.gov/Archives/edgar/data/1001171/0001140361-22-017275.txt"
val headers = mapOf("User-Agent" to "SampleCompanyName AdminContact@samplecompany.com")

In [4]:
val response = ""

In [6]:
import org.jsoup.Jsoup
import org.jsoup.nodes.Document
import java.net.HttpURLConnection
import java.net.URL

In [7]:
val connection = URL(url).openConnection() as HttpURLConnection 

In [8]:
headers.forEach { (key, value) -> connection.setRequestProperty(key, value) }

In [9]:
connection.requestMethod = "GET"
connection.connect()

In [10]:
val dF = connection.inputStream.bufferedReader().readText()

In [ ]:
println(dF)

In [11]:
val document = Jsoup.parse(dF)

In [ ]:
println(document)

In [12]:
val reportDateFocus = document.selectFirst("dei|documentfiscalperiodfocus")

In [13]:
println(reportDateFocus)

<dei:documentfiscalperiodfocus contextref="c20210101to20211231" id="Fact_c0b5474c9c3d49c596bd9b8999798287">
 FY
</dei:documentfiscalperiodfocus>


In [14]:
val reportDate = eP.parseElement(document, reportDateFocus)

In [15]:
println(reportDate)

{name=dei:documentfiscalperiodfocus, value=FY, unit=NA, date=2021-12-31}
